# Multiple Mercury

Simulate a Solar System, in a *jumping jupiter* scenario, evaluating different configurations of semi-axes for a hypothetical planet Mercury in a compact orbit.

Tasks:
+ Create directory tree
+ Create the Swift configuration files
+ Run the simulation
+ Extract figures


## Create directory tree

In [1]:
import os
import shutil
from glob import glob
import pandas as pd
import numpy as np
import math

import sys
sys.path.insert(0, 'src/')
import oe2pv

In [2]:
simulation_name = "mercury"
level1 = 10 # Number of mercury varying with distance 
level2 = 100 # Number of clones
simulation_path = "output/" + simulation_name
absolute_path = os.getcwd()

In [3]:
os.chdir("output")

if os.path.isdir(simulation_name):
    shutil.rmtree(simulation_name)
    os.mkdir(simulation_name)
    os.chdir(simulation_name)
else:
    os.mkdir(simulation_name)
    os.chdir(simulation_name)

for i in range(level1):
    for j in range(level2):
        os.mkdir(simulation_name + "-" + "{:03d}".format(i) + "-" + "{:03d}".format(j))
        
os.chdir(absolute_path)

### Check directories tree

In [4]:
!ls output/mercury/

mercury-000-000  mercury-002-050  mercury-005-000  mercury-007-050
mercury-000-001  mercury-002-051  mercury-005-001  mercury-007-051
mercury-000-002  mercury-002-052  mercury-005-002  mercury-007-052
mercury-000-003  mercury-002-053  mercury-005-003  mercury-007-053
mercury-000-004  mercury-002-054  mercury-005-004  mercury-007-054
mercury-000-005  mercury-002-055  mercury-005-005  mercury-007-055
mercury-000-006  mercury-002-056  mercury-005-006  mercury-007-056
mercury-000-007  mercury-002-057  mercury-005-007  mercury-007-057
mercury-000-008  mercury-002-058  mercury-005-008  mercury-007-058
mercury-000-009  mercury-002-059  mercury-005-009  mercury-007-059
mercury-000-010  mercury-002-060  mercury-005-010  mercury-007-060
mercury-000-011  mercury-002-061  mercury-005-011  mercury-007-061
mercury-000-012  mercury-002-062  mercury-005-012  mercury-007-062
mercury-000-013  mercury-002-063  mercury-005-013  mercury-007-063
mercury-000-014  mercury-002-064  mercury-005-014  mercury-007

## Create the Swift configuration files

### Create planets dataframe

In [5]:
# Create planets data (J2000)
# 
# Data from https://nssdc.gsfc.nasa.gov/planetary/
# Data sequence: 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus',
#  'Neptune'.

# Planet's names
planets_name = pd.Series(['mercury','venus', 'earth', 'mars', 'jupiter', \
                          'saturn', 'uranus', 'neptune'])

# Semi-axis [au]
a = pd.Series([0.38709893, 0.72333199, 1.00000011, 1.52366231, 5.20336301,\
               9.53707032, 19.19126393, 30.06896348])

e = pd.Series([0.205, 0.00677323, 0.01671022, 0.09341233, 0.04839266, 0.0541506,\
               0.04716771, 0.00858587])

i = pd.Series([7.0, 3.39471, 5e-05, 1.85061, 1.3053, 2.48446, 0.76986, 1.76917])

capom = pd.Series([1.0, 76.68069, -11.26064, 49.57854, 100.55615, 113.71504, \
                   74.22988, 131.72169])

omega = pd.Series([90.0, 131.53298, 102.94719, 336.04084, 14.75385, 92.43194, \
                   170.96424, 44.97135])


# Mass [kg]
mass = pd.Series([0.33011, 4.8675, 5.9723, 0.64171, 1898.19, 568.34, 86.813,\
                  102.413]) * 1e24

# Equatorial radius [km]
radio = pd.Series([2439.7, 6051.8, 6378.137, 3396.2, 71492, 60268, 25559, 24764])

# Period [days]
period = pd.Series([87.969, 224.701, 365.256, 686.980, 4332.589, 10759.22,\
                    30685.4, 60189])

M = pd.Series([90.0, 181.97973, 100.46435, 355.45332, 34.40438, 49.94432, 313.23218, \
               304.88003])


# Create a data frame from data series
planets = pd.DataFrame({'planets_name': planets_name, 'a': a, 'e': e, 'i': i,\
                        'capom': capom, 'omega': omega,'mass': mass, \
                        'radio': radio, 'period': period, 'M': M})

# Make planets_name index
#planets = planets.set_index('planets_name')

In [6]:
#Create new column, considering G = 1
# Mass of the Sum, in kg
mass_sun_kg = 1988500e24

# Mass of the Sun, with G = 1
mass_sun_grav = 2.959139768995959e-04

# Conic section is ellipse
ialpha = -1

# Gravitational factor of the Sun
gm =  2.959139768995959e-04

# Create mass_grav column
planets['mass_grav'] = planets.mass * mass_sun_grav / mass_sun_kg

# Create gmpl
planets['gmpl'] = gm + planets.mass_grav

In [7]:
# Que coisa linda!!!
for i in planets_name:
    exec(i + ' = planets.loc[planets.planets_name == i].drop("planets_name", 1)')

### Check planets' dataframe

In [8]:
print("\nMercury\n", mercury)
print("\nEarth\n", earth)
print("\nJupiter\n", jupiter)


Mercury
       M         a  capom      e    i          mass  omega  period   radio  \
0  90.0  0.387099    1.0  0.205  7.0  3.301100e+23   90.0  87.969  2439.7   

      mass_grav      gmpl  
0  4.912455e-11  0.000296  

Earth
            M    a     capom        e        i          mass      omega  \
2  100.46435  1.0 -11.26064  0.01671  0.00005  5.972300e+24  102.94719   

    period     radio     mass_grav      gmpl  
2  365.256  6378.137  8.887539e-10  0.000296  

Jupiter
           M         a      capom         e       i          mass     omega  \
4  34.40438  5.203363  100.55615  0.048393  1.3053  1.898190e+27  14.75385   

     period    radio     mass_grav      gmpl  
4  4332.589  71492.0  2.824747e-07  0.000296  


In [9]:
# Creating variables to use the "orbel" function
gm = planets['gmpl']
a = planets['a']
e = planets['e']
inc = planets['i']
capom = planets['capom']
omega = planets['omega']
capm = planets['M']
P = planets['period']
rpl = planets['radio']

#Create position and velocity columns
#
# The module eo2pv.so, constructed from the Swift conversion subroutine,
# will be used.

len_planets = len(planets)

x = np.zeros(len_planets)
y = np.zeros(len_planets)
z = np.zeros(len_planets)
vx = np.zeros(len_planets)
vy = np.zeros(len_planets)
vz = np.zeros(len_planets)

for j in range(len(planets)):
    x[j], y[j], z[j], vx[j], vy[j], vz[j] = oe2pv.orbel_el2xv(gm[j], ialpha, a[j],e[j],\
                                                        math.radians(inc[j]),\
                                                        math.radians(capom[j]),\
                                                        math.radians(omega[j]),\
                                                        capm[j])


# Create colums x, y, v, vx, vy and vz
planets['x'] = x
planets['y'] = y
planets['z'] = z
planets['vx'] = vx
planets['vy'] = vy
planets['vz'] = vz

print(planets)

           M          a      capom         e        i          mass  \
0   90.00000   0.387099    1.00000  0.205000  7.00000  3.301100e+23   
1  181.97973   0.723332   76.68069  0.006773  3.39471  4.867500e+24   
2  100.46435   1.000000  -11.26064  0.016710  0.00005  5.972300e+24   
3  355.45332   1.523662   49.57854  0.093412  1.85061  6.417100e+23   
4   34.40438   5.203363  100.55615  0.048393  1.30530  1.898190e+27   
5   49.94432   9.537070  113.71504  0.054151  2.48446  5.683400e+26   
6  313.23218  19.191264   74.22988  0.047168  0.76986  8.681300e+25   
7  304.88003  30.068963  131.72169  0.008586  1.76917  1.024130e+26   

       omega     period planets_name      radio     mass_grav      gmpl  \
0   90.00000     87.969      mercury   2439.700  4.912455e-11  0.000296   
1  131.53298    224.701        venus   6051.800  7.243456e-10  0.000296   
2  102.94719    365.256        earth   6378.137  8.887539e-10  0.000296   
3  336.04084    686.980         mars   3396.200  9.549457e-1

In [17]:
x = planets.iloc[4][12]
y = planets.iloc[4][13]
z = planets.iloc[4][14]
np.sqrt(x**2 + y**2 + z**2)

5.4524851578288267

0.98332821956600136

### Create mercury dataframe

In [10]:
os.chdir(absolute_path)
os.chdir("output/" + simulation_name)

simulations = glob("*")
for simulation in simulations:
    os.chdir(simulation)
    with open("pl.in", "w") as f:
        f.write("123\n456\n789")
    os.chdir("../")

os.chdir(absolute_path)

## Run the simulation

## Extract figures